#Notebook shortcuts

### shift+enter = Run cell and move to the next one

### ctrl+alt+P = insert cell above
### ctrl+alt+N = insert cell below

### ctrl+alt+up = move cell up
### ctrl+alt+down = move cell down


for more shortcuts go to https://docs.microsoft.com/en-us/azure/databricks/notebooks/notebooks-use

# Start here if the Lake has been mounted to this cluster

Check file structure is setup

In [4]:
display(dbutils.fs.ls("/mnt/coviddata/"))
        

path,name,size
dbfs:/mnt/coviddata/04-20-2020.txt,04-20-2020.txt,386595
dbfs:/mnt/coviddata/5a32172c-c259-4d99-a36b-7f69d73b844f,5a32172c-c259-4d99-a36b-7f69d73b844f,355018
dbfs:/mnt/coviddata/UID_ISO_FIPS_LookUp_Table.csv,UID_ISO_FIPS_LookUp_Table.csv,355018
dbfs:/mnt/coviddata/data_92dda754-047f-4ce5-ba65-380053b7d0a7_406e314c-6fea-457f-8550-2353d6224396.txt,data_92dda754-047f-4ce5-ba65-380053b7d0a7_406e314c-6fea-457f-8550-2353d6224396.txt,276136
dbfs:/mnt/coviddata/inputs/,inputs/,0
dbfs:/mnt/coviddata/outputs/,outputs/,0


Check all the required files are visible

In [6]:
display(dbutils.fs.ls("/mnt/coviddata/inputs/"))

path,name,size
dbfs:/mnt/coviddata/inputs/01-22-2020.csv,01-22-2020.csv,1886
dbfs:/mnt/coviddata/inputs/04-02-2020.csv,04-02-2020.csv,243956
dbfs:/mnt/coviddata/inputs/04-03-2020.csv,04-03-2020.csv,327070
dbfs:/mnt/coviddata/inputs/04-04-2020.csv,04-04-2020.csv,312226
dbfs:/mnt/coviddata/inputs/04-12-2020.csv,04-12-2020.csv,305548
dbfs:/mnt/coviddata/inputs/04-15-2020.csv,04-15-2020.csv,312551
dbfs:/mnt/coviddata/inputs/04-20-2020.csv,04-20-2020.csv,317177
dbfs:/mnt/coviddata/inputs/04-22-2020.csv,04-22-2020.csv,319302
dbfs:/mnt/coviddata/inputs/04-27-2020.csv,04-27-2020.csv,325150
dbfs:/mnt/coviddata/inputs/04-28-2020.txt,04-28-2020.txt,397693


#Wrangle the Doctor Data

In [8]:
filepath2="/mnt/coviddata/inputs/DoctorCountLatest.csv"


###Infer the schema and load the data to a spark data frame. 
### Cache the data for faster operations

In [10]:
doctorraw = spark.read.format('csv').options(header='false', inferSchema='true', comment='I').load(filepath2)
doctorraw.cache()

Out[10]: DataFrame[_c0: int, _c1: string, _c2: string, _c3: string, _c4: string, _c5: int, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: double, _c15: double, _c16: string, _c17: string, _c18: string, _c19: string, _c20: int, _c21: string, _c22: string]

### Check if schema was inferred correctly

In [12]:
doctorraw.printSchema()

root
-- _c0: integer (nullable = true)
-- _c1: string (nullable = true)
-- _c2: string (nullable = true)
-- _c3: string (nullable = true)
-- _c4: string (nullable = true)
-- _c5: integer (nullable = true)
-- _c6: string (nullable = true)
-- _c7: string (nullable = true)
-- _c8: string (nullable = true)
-- _c9: string (nullable = true)
-- _c10: string (nullable = true)
-- _c11: string (nullable = true)
-- _c12: string (nullable = true)
-- _c13: string (nullable = true)
-- _c14: double (nullable = true)
-- _c15: double (nullable = true)
-- _c16: string (nullable = true)
-- _c17: string (nullable = true)
-- _c18: string (nullable = true)
-- _c19: string (nullable = true)
-- _c20: integer (nullable = true)
-- _c21: string (nullable = true)
-- _c22: string (nullable = true)

### Display the data in a nice readable format

In [14]:
display(doctorraw)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22
22092804,HWF_0001,COUNTRY,AGO,YEAR,1997,null,null,null,null,null,null,null,null,0.592,0.592,null,null,null,2020-02-12 09:20:25.0000000 +00:00,1997,1996-12-31 23:00:00.0000000 +00:00,1997-12-30 23:00:00.0000000 +00:00
22092805,HWF_0001,COUNTRY,AGO,YEAR,2004,null,null,null,null,null,null,null,null,0.621,0.621,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2004,2003-12-31 23:00:00.0000000 +00:00,2004-12-30 23:00:00.0000000 +00:00
22092806,HWF_0001,COUNTRY,AGO,YEAR,2009,null,null,null,null,null,null,null,null,1.313,1.313,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2009,2008-12-31 23:00:00.0000000 +00:00,2009-12-30 23:00:00.0000000 +00:00
22092807,HWF_0001,COUNTRY,AGO,YEAR,2017,null,null,null,null,null,null,null,null,2.146,2.146,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2017,2016-12-31 23:00:00.0000000 +00:00,2017-12-30 23:00:00.0000000 +00:00
22092808,HWF_0001,COUNTRY,BDI,YEAR,2004,null,null,null,null,null,null,null,null,0.28,0.28,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2004,2003-12-31 23:00:00.0000000 +00:00,2004-12-30 23:00:00.0000000 +00:00
22092809,HWF_0001,COUNTRY,BDI,YEAR,2010,null,null,null,null,null,null,null,null,0.482,0.482,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2010,2009-12-31 23:00:00.0000000 +00:00,2010-12-30 23:00:00.0000000 +00:00
22092810,HWF_0001,COUNTRY,BDI,YEAR,2011,null,null,null,null,null,null,null,null,0.399,0.399,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2011,2010-12-31 23:00:00.0000000 +00:00,2011-12-30 23:00:00.0000000 +00:00
22092811,HWF_0001,COUNTRY,BDI,YEAR,2012,null,null,null,null,null,null,null,null,0.539,0.539,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2012,2011-12-31 23:00:00.0000000 +00:00,2012-12-30 23:00:00.0000000 +00:00
22092812,HWF_0001,COUNTRY,BDI,YEAR,2013,null,null,null,null,null,null,null,null,0.581,0.581,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2013,2012-12-31 23:00:00.0000000 +00:00,2013-12-30 23:00:00.0000000 +00:00
22092813,HWF_0001,COUNTRY,BDI,YEAR,2014,null,null,null,null,null,null,null,null,0.619,0.619,null,null,null,2020-02-12 09:20:25.0000000 +00:00,2014,2013-12-31 23:00:00.0000000 +00:00,2014-12-30 23:00:00.0000000 +00:00


### focus on the columns we want to work with

In [16]:
display(doctorraw.select("_c3","_c5","_c15"))

_c3,_c5,_c15
AGO,1997,0.592
AGO,2004,0.621
AGO,2009,1.313
AGO,2017,2.146
BDI,2004,0.28
BDI,2010,0.482
BDI,2011,0.399
BDI,2012,0.539
BDI,2013,0.581
BDI,2014,0.619


In [17]:
doctorraw.select("_c3","_c5","_c15").show()

+---+----+-----+
_c3| _c5| _c15|
+---+----+-----+
AGO|1997|0.592|
AGO|2004|0.621|
AGO|2009|1.313|
AGO|2017|2.146|
BDI|2004| 0.28|
BDI|2010|0.482|
BDI|2011|0.399|
BDI|2012|0.539|
BDI|2013|0.581|
BDI|2014|0.619|
BDI|2015| 0.5|
BDI|2017|1.001|
BEN|2004|0.401|
BEN|2008|0.623|
BEN|2010|0.558|
BEN|2011|0.577|
BEN|2012|1.646|
BEN|2013|1.849|
BEN|2016|0.481|
BEN|2018|0.791|
+---+----+-----+
only showing top 20 rows

### filter the data so that there is only the doctor per 10k count for the most recent year for each country in the list. We only need the latest year.

In [19]:
doctorlatest=doctorraw.groupBy("_c3").max("_c5","_c15")

### just confirm that Australia exists in the data set we're pulling

In [21]:
doctorlatest.filter("_c3= 'AUS'").show()

+---+--------+---------+
_c3|max(_c5)|max(_c15)|
+---+--------+---------+
AUS| 2017| 36.778|
+---+--------+---------+

In [22]:
display(doctorlatest)

_c3,max(_c5),max(_c15)
NIU,2008,23.529
HTI,2018,2.384
BRB,2017,24.843
LVA,2017,46.934
POL,2017,24.188
ZMB,2018,11.867
JAM,2017,13.061
BRA,2018,21.652
ARM,2017,44.023
MOZ,2018,0.838


### rename the columns

In [24]:
doctorlatest=doctorlatest.withColumnRenamed("_c3",'COUNTRY').withColumnRenamed("max(_c5)",'YEAR').withColumnRenamed("max(_c15)",'DoctorsPer10k')

In [25]:
doctorlatest.printSchema()

root
-- COUNTRY: string (nullable = true)
-- YEAR: integer (nullable = true)
-- DoctorsPer10k: double (nullable = true)

In [26]:
doctorlatest.printSchema()

root
-- COUNTRY: string (nullable = true)
-- YEAR: integer (nullable = true)
-- DoctorsPer10k: double (nullable = true)

### save it to csv on our data lake

In [28]:
#doctorlatest.write.mode('overwrite').option("header","true").csv('/mnt/coviddata/outputs/DoctorCountLatestYear')

###Load the country code data

In [30]:
filepath3="/mnt/coviddata/inputs/UID_ISO_FIPS_LookUp_Table.csv"

In [31]:
countrycodes = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath3)

In [32]:
countrycodes.printSchema()

root
-- UID: integer (nullable = true)
-- iso2: string (nullable = true)
-- iso3: string (nullable = true)
-- code3: integer (nullable = true)
-- FIPS: integer (nullable = true)
-- Admin2: string (nullable = true)
-- Province_State: string (nullable = true)
-- Country_Region: string (nullable = true)
-- Lat: double (nullable = true)
-- Long_: double (nullable = true)
-- Combined_Key: string (nullable = true)
-- Population: integer (nullable = true)

In [33]:
countrycodes.show()

+---+----+----+-----+----+------+--------------+-------------------+---------+----------+-------------------+----------+
UID|iso2|iso3|code3|FIPS|Admin2|Province_State| Country_Region| Lat| Long_| Combined_Key|Population|
+---+----+----+-----+----+------+--------------+-------------------+---------+----------+-------------------+----------+
 4| AF| AFG| 4|null| null| null| Afghanistan| 33.93911| 67.709953| Afghanistan| 38928341|
 8| AL| ALB| 8|null| null| null| Albania| 41.1533| 20.1683| Albania| 2877800|
 12| DZ| DZA| 12|null| null| null| Algeria| 28.0339| 1.6596| Algeria| 43851043|
 20| AD| AND| 20|null| null| null| Andorra| 42.5063| 1.5218| Andorra| 77265|
 24| AO| AGO| 24|null| null| null| Angola| -11.2027| 17.8739| Angola| 32866268|
 28| AG| ATG| 28|null| null| null|Antigua and Barbuda| 17.0608| -61.7964|Antigua and Barbuda| 97928|
 32| AR| ARG| 32|null| null| null| Argentina| -38.4161| -63.6167| Argentina| 45195777|
 51| AM| ARM| 51|null| null| null| Armenia| 40.0691| 45.0382| Armenia| 2963234|
 40| AT| AUT| 40|null| null| null| Austria| 47.5162| 14.5501| Austria| 9006400|
 31| AZ| AZE| 31|null| null| null| Azerbaijan| 40.1431| 47.5769| Azerbaijan| 10139175|
 44| BS| BHS| 44|null| null| null| Bahamas|25.025885|-78.035889| Bahamas| 393248|
 48| BH| BHR| 48|null| null| null| Bahrain| 26.0275| 50.55| Bahrain| 1701583|
 50| BD| BGD| 50|null| null| null| Bangladesh| 23.685| 90.3563| Bangladesh| 164689383|
 52| BB| BRB| 52|null| null| null| Barbados| 13.1939| -59.5432| Barbados| 287371|
112| BY| BLR| 112|null| null| null| Belarus| 53.7098| 27.9534| Belarus| 9449321|
 56| BE| BEL| 56|null| null| null| Belgium| 50.8333| 4.469936| Belgium| 11589616|
 84| BZ| BLZ| 84|null| null| null| Belize| 17.1899| -88.4976| Belize| 397621|
204| BJ| BEN| 204|null| null| null| Benin| 9.3077| 2.3158| Benin| 12123198|
 64| BT| BTN| 64|null| null| null| Bhutan| 27.5142| 90.4336| Bhutan| 771612|
 68| BO| BOL| 68|null| null| null| Bolivia| -16.2902| -63.5887| Bolivia| 11673029|
+---+----+----+-----+----+------+--------------+-------------------+---------+----------+-------------------+----------+
only showing top 20 rows

### just want country region and iso3

In [35]:
countrycodeiso3=countrycodes.select("iso3","Country_Region").distinct()

In [36]:
countrycodeiso3.filter("iso3='AUS'").show()

+----+--------------+
iso3|Country_Region|
+----+--------------+
 AUS| Australia|
+----+--------------+

In [37]:
#countrycodeiso3.write.mode('overwrite').csv('/mnt/coviddata/outputs/CountryCodesISO3')

#Wrangle the COVID Data

### load the covid data and summarize by country
### join the summarized data with the count of doctors and country codes

In [39]:
# Creating widgets for leveraging parameters, and printing the parameters

dbutils.widgets.text("input", "","")
y = dbutils.widgets.get("input")
print ("Param -\'input':")
print (y)

dbutils.widgets.text("fileDate", "","")
z = dbutils.widgets.get("fileDate")
print ("Param -\'fileDate':")
print (z)

dbutils.widgets.text("name", "","")
u = dbutils.widgets.get("name")
print ("Param -\'name':")
print (u)

dbutils.widgets.text("name2", "","")
v = dbutils.widgets.get("name2")
print ("Param -\'name2':")
print (v)

Param -'input':

Param -'fileDate':

Param -'name':

Param -'name2':

In [40]:
filepath = "/mnt/coviddata/inputs/latestcovidcount.csv"

### infer the schema and load the data into a spark dataframe

In [42]:
covidraw = spark.read.format('csv').options(header='true', inferSchema='true').load(filepath)


In [43]:
display(covidraw)

FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
45001,Abbeville,South Carolina,US,2020-08-02T04:34:47.000+0000,34.22333378,-82.46170658,288,7,0,281,"Abbeville, South Carolina, US",1174.2161699351734,2.4305555555555554
22001,Acadia,Louisiana,US,2020-08-02T04:34:47.000+0000,30.295064899999996,-92.41419698,2331,71,0,2260,"Acadia, Louisiana, US",3756.950600370699,3.045903045903046
51001,Accomack,Virginia,US,2020-08-02T04:34:47.000+0000,37.76707161,-75.63234615,1077,15,0,1062,"Accomack, Virginia, US",3332.7144448570366,1.392757660167131
16001,Ada,Idaho,US,2020-08-02T04:34:47.000+0000,43.4526575,-116.24155159999998,8004,62,0,7942,"Ada, Idaho, US",1662.0049959820346,0.7746126936531734
19001,Adair,Iowa,US,2020-08-02T04:34:47.000+0000,41.33075609,-94.47105874,20,0,0,20,"Adair, Iowa, US",279.6420581655481,0.0
21001,Adair,Kentucky,US,2020-08-02T04:34:47.000+0000,37.10459774,-85.28129668,201,19,0,182,"Adair, Kentucky, US",1046.765961878971,9.45273631840796
29001,Adair,Missouri,US,2020-08-02T04:34:47.000+0000,40.19058551,-92.60078167,134,0,0,134,"Adair, Missouri, US",528.7456102276764,0.0
40001,Adair,Oklahoma,US,2020-08-02T04:34:47.000+0000,35.88494195,-94.65859267,289,5,0,284,"Adair, Oklahoma, US",1302.1537352437595,1.7301038062283738
8001,Adams,Colorado,US,2020-08-02T04:34:47.000+0000,39.87432092,-104.3362578,6018,171,0,5847,"Adams, Colorado, US",1163.0761024388264,2.8414755732801593
16003,Adams,Idaho,US,2020-08-02T04:34:47.000+0000,44.89333571,-116.4545247,18,0,0,18,"Adams, Idaho, US",419.18956683744767,0.0


In [44]:
covidlatest=covidraw.select("Country_Region","Confirmed","Deaths","Recovered").groupby("Country_Region").sum("Confirmed","Deaths","Recovered")

In [45]:
display(covidlatest)

Country_Region,sum(Confirmed),sum(Deaths),sum(Recovered)
Chad,936,75,813
Russia,843890,14034,645316
Paraguay,5485,52,3786
Yemen,1730,494,862
Senegal,10284,209,6822
Sweden,80422,5743,0
Cabo Verde,2480,24,1837
Guyana,430,20,185
Burma,353,6,297
Eritrea,279,0,225


In [46]:
#covidlatest.write.mode('overwrite').csv('/mnt/coviddata/outputs/CovidLatest')

In [47]:
doctorlatest.show()

+-------+----+-------------+
COUNTRY|YEAR|DoctorsPer10k|
+-------+----+-------------+
 NIU|2008| 23.529|
 HTI|2018| 2.384|
 BRB|2017| 24.843|
 LVA|2017| 46.934|
 POL|2017| 24.188|
 ZMB|2018| 11.867|
 JAM|2017| 13.061|
 BRA|2018| 21.652|
 ARM|2017| 44.023|
 MOZ|2018| 0.838|
 CUB|2018| 84.218|
 JOR|2017| 28.078|
 SOM|2014| 0.279|
 FRA|2018| 34.41|
 COD|2016| 0.934|
 BRN|2017| 17.812|
 FSM|2009| 6.184|
 BOL|2016| 15.901|
 URY|2017| 50.794|
 LBY|2017| 20.905|
+-------+----+-------------+
only showing top 20 rows

In [48]:
countrycodeiso3.show()

+----+------------------+
iso3| Country_Region|
+----+------------------+
 GNQ| Equatorial Guinea|
 LBY| Libya|
 ASM| US|
 HTI| Haiti|
 AUT| Austria|
 PYF| France|
 ECU| Ecuador|
 JAM| Jamaica|
 RWA| Rwanda|
 UZB| Uzbekistan|
 DOM|Dominican Republic|
 MEX| Mexico|
 LBR| Liberia|
 BMU| United Kingdom|
 COL| Colombia|
 YEM| Yemen|
 GMB| Gambia|
 ZWE| Zimbabwe|
 SVN| Slovenia|
 SUR| Suriname|
+----+------------------+
only showing top 20 rows

In [49]:
from pyspark.sql.functions import col
doctoriso3=doctorlatest.join(countrycodeiso3,col("COUNTRY")==col("iso3"))

In [50]:
doctoriso3.select("COUNTRY","YEAR","DoctorsPer10k","Country_Region").filter("COUNTRY = 'AUS'").show()

+-------+----+-------------+--------------+
COUNTRY|YEAR|DoctorsPer10k|Country_Region|
+-------+----+-------------+--------------+
 AUS|2017| 36.778| Australia|
+-------+----+-------------+--------------+

In [51]:
coviddoctors = covidlatest.join(doctoriso3, doctoriso3.Country_Region == covidlatest.Country_Region)

In [52]:
coviddoctors.show()

+------------------+--------------+-----------+--------------+-------+----+-------------+----+------------------+
 Country_Region|sum(Confirmed)|sum(Deaths)|sum(Recovered)|COUNTRY|YEAR|DoctorsPer10k|iso3| Country_Region|
+------------------+--------------+-----------+--------------+-------+----+-------------+----+------------------+
 Equatorial Guinea| 4821| 83| 2182| GNQ|2017| 4.017| GNQ| Equatorial Guinea|
 Libya| 3691| 80| 619| LBY|2017| 20.905| LBY| Libya|
 Haiti| 7468| 165| 4606| HTI|2018| 2.384| HTI| Haiti|
 Austria| 21212| 718| 18911| AUT|2017| 51.697| AUT| Austria|
 Ecuador| 86232| 5736| 36213| ECU|2016| 21.111| ECU| Ecuador|
 Jamaica| 883| 10| 743| JAM|2017| 13.061| JAM| Jamaica|
 Rwanda| 2042| 5| 1119| RWA|2018| 1.371| RWA| Rwanda|
 Uzbekistan| 24783| 147| 15299| UZB|2014| 33.911| UZB| Uzbekistan|
Dominican Republic| 71415| 1170| 37509| DOM|2011| 18.498| DOM|Dominican Republic|
 Mexico| 434193| 47472| 334867| MEX|2017| 23.827| MEX| Mexico|
 Liberia| 1189| 75| 673| LBR|2015| 0.376| LBR| Liberia|
 Colombia| 306181| 10330| 160708| COL|2018| 21.848| COL| Colombia|
 Yemen| 1730| 494| 862| YEM|2014| 5.251| YEM| Yemen|
 Gambia| 498| 9| 68| GMB|2015| 1.109| GMB| Gambia|
 Zimbabwe| 3659| 69| 1011| ZWE|2018| 2.096| ZWE| Zimbabwe|
 Slovenia| 2171| 119| 1821| SVN|2017| 30.861| SVN| Slovenia|
 Suriname| 1760| 26| 1161| SUR|2018| 12.101| SUR| Suriname|
 Serbia| 25882| 582| 0| SRB|2016| 31.131| SRB| Serbia|
 Bahamas| 599| 14| 91| BHS|2017| 27.553| BHS| Bahamas|
 Slovakia| 2337| 29| 1742| SVK|2017| 34.664| SVK| Slovakia|
+------------------+--------------+-----------+--------------+-------+----+-------------+----+------------------+
only showing top 20 rows

In [53]:
coviddoctorselect = coviddoctors.select(covidlatest.Country_Region,"sum(Confirmed)","sum(Deaths)","sum(Recovered)","COUNTRY","YEAR","DoctorsPer10k")

In [54]:
coviddoctorfinal=coviddoctorselect\
.withColumnRenamed('sum(Confirmed)','Confirmed')\
.withColumnRenamed('sum(Deaths)','Deaths')\
.withColumnRenamed('sum(Recovered)','Recovered')\
.withColumnRenamed('COUNTRY','Iso3')\
.withColumnRenamed('YEAR','YearOfDoctorCount')

In [55]:
coviddoctorfinal.printSchema()

root
-- Country_Region: string (nullable = true)
-- Confirmed: long (nullable = true)
-- Deaths: long (nullable = true)
-- Recovered: long (nullable = true)
-- Iso3: string (nullable = true)
-- YearOfDoctorCount: integer (nullable = true)
-- DoctorsPer10k: double (nullable = true)

In [56]:
coviddoctorfinal.write.mode("overwrite").option("header", "true").csv("/mnt/coviddata/outputs/CovidDoctorCombined")

In [57]:
#dbutils.fs.rm("/mnt/coviddata/outputs/final",True)
#dbutils.fs.mkdirs("/mnt/coviddata/outputs/final")

### define where you want your final data to go

In [59]:
dbutils.fs.rm("/mnt/coviddata/outputs/finalVJ",True)
dbutils.fs.mkdirs("/mnt/coviddata/outputs/finalVJ")

Out[47]: True

### for the data factory to correctly copy these shards to Synapse, we need to remove any unneeded files from the output directory. Since trying to find files that begin with "_" seem to throw java, we will look for all the .csv files and move them to a clean directory and then point the factory to that as the source

In [61]:
%scala

//CHANGE THIS FOLDER DEPENDING ON YOUR DATA
val fileprefix= "/mnt/coviddata/outputs/finalVJ/"
val partition_path = dbutils.fs.ls("/mnt/coviddata/outputs/CovidDoctorCombined")
     .filter(file=>file.name.endsWith("csv"))//(0).path

partition_path.foreach { file => dbutils.fs.cp(file.path,fileprefix+file.name)}

//partition_path.show()

//partition_path.toDF().foreach { file => dbutils.fs.cp(file(0).toString,)}//.toString, true)}


//dbutils.fs.cp(partition_path,fileprefix+".tab")

//dbutils.fs.rm(fileprefix+".tmp",recurse=true)

fileprefix: String = /mnt/coviddata/outputs/finalVJ/
partition_path: Seq[com.databricks.backend.daemon.dbutils.FileInfo] = WrappedArray(FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00000-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4894-1-c000.csv, part-00000-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4894-1-c000.csv, 125), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00002-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4896-1-c000.csv, part-00002-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4896-1-c000.csv, 113), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00003-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4897-1-c000.csv, part-00003-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4897-1-c000.csv, 114), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00004-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4898-1-c000.csv, part-00004-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4898-1-c000.csv, 119), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00009-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4903-1-c000.csv, part-00009-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4903-1-c000.csv, 189), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00010-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4904-1-c000.csv, part-00010-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4904-1-c000.csv, 122), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00011-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4905-1-c000.csv, part-00011-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4905-1-c000.csv, 174), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00012-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4906-1-c000.csv, part-00012-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4906-1-c000.csv, 114), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00016-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4910-1-c000.csv, part-00016-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4910-1-c000.csv, 124), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00017-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4911-1-c000.csv, part-00017-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4911-1-c000.csv, 113), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00018-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4912-1-c000.csv, part-00018-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4912-1-c000.csv, 147), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00019-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4913-1-c000.csv, part-00019-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4913-1-c000.csv, 156), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00020-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4914-1-c000.csv, part-00020-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4914-1-c000.csv, 114), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00021-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4915-1-c000.csv, part-00021-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4915-1-c000.csv, 113), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00023-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4917-1-c000.csv, part-00023-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4917-1-c000.csv, 117), FileInfo(dbfs:/mnt/coviddata/outputs/CovidDoctorCombined/part-00024-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4918-1-c000.csv, part-00024-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4918-1-c000.csv, 122), FileInfo

In [62]:
display(dbutils.fs.ls('/mnt/coviddata/outputs/finalVJ/'))

path,name,size
dbfs:/mnt/coviddata/outputs/finalVJ/part-00000-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4894-1-c000.csv,part-00000-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4894-1-c000.csv,125
dbfs:/mnt/coviddata/outputs/finalVJ/part-00002-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4896-1-c000.csv,part-00002-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4896-1-c000.csv,113
dbfs:/mnt/coviddata/outputs/finalVJ/part-00003-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4897-1-c000.csv,part-00003-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4897-1-c000.csv,114
dbfs:/mnt/coviddata/outputs/finalVJ/part-00004-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4898-1-c000.csv,part-00004-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4898-1-c000.csv,119
dbfs:/mnt/coviddata/outputs/finalVJ/part-00009-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4903-1-c000.csv,part-00009-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4903-1-c000.csv,189
dbfs:/mnt/coviddata/outputs/finalVJ/part-00010-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4904-1-c000.csv,part-00010-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4904-1-c000.csv,122
dbfs:/mnt/coviddata/outputs/finalVJ/part-00011-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4905-1-c000.csv,part-00011-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4905-1-c000.csv,174
dbfs:/mnt/coviddata/outputs/finalVJ/part-00012-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4906-1-c000.csv,part-00012-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4906-1-c000.csv,114
dbfs:/mnt/coviddata/outputs/finalVJ/part-00016-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4910-1-c000.csv,part-00016-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4910-1-c000.csv,124
dbfs:/mnt/coviddata/outputs/finalVJ/part-00017-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4911-1-c000.csv,part-00017-tid-8261168128314198120-d20532ca-c7cf-4cd5-9bfc-eb83d87768af-4911-1-c000.csv,113
